# Slice Builder: GraphML Parsing

This works in tandem with the Graph Analyzer code previously written, if desired (not required). That code can be found here: https://github.com/pjw7904/Graph-Analyzer/tree/develop

Any graphml file comprised of basic node and edge tags will be able to work with this parser. Each node will be set up as a FABRIC node and any edge will be set up as a l2network between the specified nodes. Currently, this code does not consider extended LANs with more than two nodes.

This was written to take advantage of existing graphml files, as opposed to the FABRIC-enhanced graphml RSPEC files that contain hardware properties that go beyond the basic topological information.

## Input Required Information

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of slice you want to create. Please make sure a slice with that name does not already exist. |
| SITE_NAME     | Name of the FABRIC site you want the nodes to be reserved at. This code does not consider inter-site situations, the entire topology is reserved on a single slice. |
| GRAPH_PATH    | Path to the graphml file you want to use to create a topology. |
| HAS_CLIENTS   | Enter True if clients are present in topology, if not, False. These nodes and the networks connecting them utilizes alternative naming and addressing structures. |
| CLIENT_PREFIX | The naming prefix given to each node (currently, this is required if the topology does have clients) |
| MEAS_ADD      | Enter True if measurements are to be taken on the slice. This requires the inclusion of a separate measurement node |

In [ ]:
SLICE_NAME = "test"
SITE_NAME = "WASH"
GRAPH_PATH = "/home/fabric/work/custom/FABRIC-Automation/graphs/triangle.graphml"
HAS_CLIENTS = False
CLIENT_PREFIX = "ipnode"
MEAS_ADD = True

## Import the FABlib Library and Confirm the Configuration is Correct

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Parse the GraphML for the Topology and Create the Slice

The minidom library is used to parse the graphml. It assumes proper use of the node and edge tags. Examples of valid tags can be seen below. The name of the node is the node's id. Furthermore, it does not matter which node is the source and destination, it is parsed as an undirected graph and there will be issues if you try and create another network between the same two nodes.

    <node id="L1" />
    <node id="S1" />
    <edge source="L1" target="S1" />


In [ ]:
import xml.dom.minidom
from collections import Counter

try:        
    #Create the slice
    slice = fablib.new_slice(name=SLICE_NAME)
    
    # Create dictionary to store nodes
    nodeDict = {}
    
    # Create dictionary to store information to dump into a file
    logFile = {"name": SLICE_NAME, "site": SITE_NAME}
    
    # Use XML parser to parse the GraphML file
    docs = xml.dom.minidom.parse(GRAPH_PATH)

    # Find all nodes via the node tag, add each to the slice with Rocky Linux as its base
    nodes = docs.getElementsByTagName("node")
    for node in nodes:
        # Grab the node name and determine if it is a client node based on its name prefix
        nodeName = node.getAttribute("id")
        isClient = True if HAS_CLIENTS and nodeName.startswith(CLIENT_PREFIX) else False # Check for compute/client nodes
        
        ### LOG FILE INFO
        logFile[nodeName] = {"isClient": isClient, "networks": {}}
        
        # Add node to the slice
        nodeInfo = slice.add_node(name=nodeName, cores=1, ram=4, image='default_rocky_8', site=SITE_NAME)
        
        # nodeDict = 0 -> FABRIC node object, 1 -> is a client/server (True) or not (False)
        nodeDict[nodeName] = {"nodeInfo": nodeInfo, "isClient": isClient}
        
        print(f'Added node {nodeName}')
    
    # Find all edges via the edge tag, add each to the slice via an L2Bridge connecting the node interfaces
    edges = docs.getElementsByTagName("edge")
    for edge in edges:
        # grab nodes x and y in edge (x,y)
        source = edge.getAttribute("source")
        target = edge.getAttribute("target")
        
        # Create an interface name for each interface in the network
        sourceIntfName = f"intf-{target}"
        targetIntfName = f"intf-{source}"
        
        # Name a network based on if it is a user-facing LAN (edge) or P2P links in the core of the network (core)
        networkPrefix = "edge" if nodeDict[source]["isClient"]  or nodeDict[target]["isClient"] else "core"
        networkName = f'{networkPrefix}-{source}-{target}'
        
        # Add a NIC for each node that is a part of the edge
        sourceIntf = nodeDict[source]["nodeInfo"].add_component(model='NIC_Basic', name=sourceIntfName).get_interfaces()[0]
        targetIntf = nodeDict[target]["nodeInfo"].add_component(model='NIC_Basic', name=targetIntfName).get_interfaces()[0]

        # Add a L2 network between the interfaces
        slice.add_l2network(name=networkName, interfaces=[sourceIntf, targetIntf], type="L2Bridge")
        
        ### LOG FILE INFO
        logFile[source]["networks"][networkName] = {"neighbor": target}
        logFile[target]["networks"][networkName] = {"neighbor": source}
        
        print(f'Added edge {source}-{target}')

except Exception as e:
    print(f"Exception: {e}")

## Add a Measurement Node (Optional)

In [ ]:
if(MEAS_ADD):
    import mflib 
    print(f"MFLib version  {mflib.__version__} " )

    from mflib.mflib import MFLib

    # Add measurement node to topology using static method.
    MFLib.addMeasNode(slice, disk=100, image='docker_ubuntu_20', site=SITE_NAME)
    print("Measurement node added.")

## Submit the Slice

In [ ]:
%%time
import json 

try:
    # Submit Slice Request
    print(f'Submitting the new slice, "{SLICE_NAME}"...')
    slice.submit()
    print(f'{SLICE_NAME} creation done.')

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

## Initalize the Measurement Framework (Optional)

This step both initalizes and instrumentizes the Measurement framework to use Prometheus and Grafana. If ELK is desired, modifications to the cell need to be made.

In [ ]:
if(MEAS_ADD):
    mf = MFLib(SLICE_NAME) # Initalize
    instrumetize_results = mf.instrumentize( ["prometheus"] ) # Instrumentize
    
    # Grafana SSH Tunnel Command
    # mf.grafana_tunnel_local_port = 10010 # optionally change the port
    print(mf.grafana_tunnel)
    print(f"Browse to https://localhost:{mf.grafana_tunnel_local_port}/grafana/dashboards?query=%2A")

## Add Basic IPv4 Addressing

In this system, 192.168.0.0/16 is the address space for all interfaces on the FABRIC slice. 

Each network is a /24 subnet of this network. Edge networks have client/compute devices with lower address (ex: .1) and networking nodes with higher addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

def updateMeasNetworkName(node, nodeName, intfName):
        if("meas" not in nodeName): 
            node.execute(command=f"sudo ip link set dev {intfName} down")
            node.execute(command=f"sudo ip link set dev {intfName} name meas")
            node.execute(command=f"sudo ip link set dev meas up")
            
            print(f"\t{nodeName} {intfName} renamed meas")
        else:
            print(f"\tMeasurement node not modified")

        return

# Start with a 1 in the third octet
thirdOctet = 1

# For each newtork in the slice
for network in slice.get_networks():
    # Grab all of the usable host addresses for a new network (new third octet).
    networkAddress = f'192.168.{thirdOctet}.0/24'
    currentIPNetwork = IPv4Network(networkAddress)
    hostIPList = list(currentIPNetwork)[1:-1] # 1:-1 = remove network and broadcast address
    getNewNetwork = False
    
    # Determine if the current network is an edge or core network
    networkName = network.get_name()
    
    isEdgeNetwork = True if networkName.startswith("edge") else False
    
    # For each interface in the network
    for intf in network.get_interfaces():        
        intfName = intf.get_physical_os_interface_name()
        node = intf.get_node()
        nodeName = node.get_name()
        
        # Measurement network is auto-assigned IP addressing, it needs intf naming updates
        if("meas" in networkName):
            updateMeasNetworkName(node, nodeName, intfName)
        else:
            print(f"Configuring network {networkName} with IPv4 Network {networkAddress}")
            # If this is an edge network and it is a networking node, not the client node, give it a high-range address.
            if(isEdgeNetwork and not nodeDict[nodeName]["isClient"]):
                currentIPv4Address = hostIPList.pop() # Highest available host address in network
            else:
                currentIPv4Address = hostIPList.pop(0) # Lowest available host address in network

            # Add the address to the node
            intf.ip_addr_add(addr=currentIPv4Address, subnet=currentIPNetwork)
            print(f"\t{nodeName} {intf.get_device_name()} = {currentIPv4Address}")
            
            getNewNetwork = True
        
            ### LOG FILE INFO
            logFile[nodeName]["networks"][networkName]["subnet"] = str(currentIPNetwork)
            logFile[nodeName]["networks"][networkName]["ipv4"] = str(currentIPv4Address)
    
    if(getNewNetwork):
        thirdOctet += 1

## Log Topology Information

In [ ]:
## LOG FILE INFO
with open("topo_log.json", "w") as outfile: 
    json.dump(logFile, outfile)